In [3]:
import numpy as np
import pandas as pd
import torch

### PLAN

Get percentages for serve, double fault, ace, break point, percentages.  
Write function to pull data for a player for the last 3 years  
Write a function to compare a player to another in their last 3 years  
Separate data by court surface  
Get data by vs righties vs righties  
Get statistic for break points conversion/saved + tiebreaks  
Get statistic for serve returns  
Get statistic for service  



In [6]:
atp_singles = 'data/atp_singles/atp_matches_'

In [12]:
atp_2024 = pd.read_csv(f'{atp_singles}2024.csv')
print(atp_2024.columns)

Index(['tourney_id', 'tourney_name', 'surface', 'draw_size', 'tourney_level',
       'tourney_date', 'match_num', 'winner_id', 'winner_seed', 'winner_entry',
       'winner_name', 'winner_hand', 'winner_ht', 'winner_ioc', 'winner_age',
       'loser_id', 'loser_seed', 'loser_entry', 'loser_name', 'loser_hand',
       'loser_ht', 'loser_ioc', 'loser_age', 'score', 'best_of', 'round',
       'minutes', 'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon',
       'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df', 'l_svpt',
       'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced',
       'winner_rank', 'winner_rank_points', 'loser_rank', 'loser_rank_points'],
      dtype='object')


In [13]:
atp_2024.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,2024-0339,Brisbane,Hard,32,A,20240101,300,105777,2,NaN,...,58.0,44.0,16.0,11.0,8.0,9.0,14.0,2570.0,8.0,3660.0
1,2024-0339,Brisbane,Hard,32,A,20240101,299,208029,1,NaN,...,35.0,31.0,10.0,11.0,5.0,7.0,8.0,3660.0,39.0,1122.0
2,2024-0339,Brisbane,Hard,32,A,20240101,298,105777,2,NaN,...,39.0,24.0,14.0,10.0,5.0,7.0,14.0,2570.0,55.0,902.0
3,2024-0339,Brisbane,Hard,32,A,20240101,297,208029,1,NaN,...,51.0,31.0,16.0,10.0,3.0,5.0,8.0,3660.0,116.0,573.0
4,2024-0339,Brisbane,Hard,32,A,20240101,296,126128,NaN,NaN,...,37.0,27.0,16.0,10.0,5.0,8.0,39.0,1122.0,44.0,1021.0


In [42]:
### Convert to Percentages
w_ace_perc = atp_2024['w_ace'] / atp_2024['w_svpt']
w_df_perc = atp_2024['w_df'] / (atp_2024['w_svpt'] - atp_2024['w_1stIn'])
w_first_in = atp_2024['w_1stIn'] / atp_2024['w_svpt']
w_first_win_perc = atp_2024['w_1stWon'] / atp_2024['w_1stIn']
w_second_in = 1 - w_df_perc
w_second_win_perc = atp_2024['w_2ndWon'] / (atp_2024['w_svpt'] - atp_2024['w_1stIn'])
w_bp_hold_perc = atp_2024['w_bpSaved'] / atp_2024['w_bpFaced']
w_bp_win_perc = 1 - (atp_2024['l_bpSaved'] / atp_2024['l_bpFaced'])
w_return_win_perc = (atp_2024['l_bpFaced'] - atp_2024['l_bpSaved']) / atp_2024['l_SvGms']

l_ace_perc = atp_2024['l_ace'] / atp_2024['l_svpt']
l_df_perc = atp_2024['l_df'] / (atp_2024['l_svpt'] - atp_2024['l_1stIn'])
l_first_in = atp_2024['l_1stIn'] / atp_2024['l_svpt']
l_first_win_perc = atp_2024['l_1stWon'] / atp_2024['l_1stIn']
l_second_in = 1 - l_df_perc
l_second_win_perc = atp_2024['l_2ndWon'] / (atp_2024['l_svpt'] - atp_2024['l_1stIn'])
l_bp_hold_perc = atp_2024['l_bpSaved'] / atp_2024['l_bpFaced']
l_bp_win_perc = 1 - w_bp_hold_perc
l_return_win_perc = (atp_2024['w_bpFaced'] - atp_2024['w_bpSaved']) / atp_2024['w_SvGms']

In [46]:
for score in atp_2024['score']:
    print(score)

7-6(5) 6-4
6-4 7-6(0)
6-3 7-5
6-2 7-6(6)
7-6(4) 6-2
5-7 7-6(6) 6-3
6-1 6-4
6-4 5-7 6-2
4-6 6-1 7-6(2)
6-7(9) 6-4 7-6(3)
6-4 3-6 7-5
6-1 6-2
W/O
5-7 6-2 7-6(4)
6-1 6-2
4-6 6-4 6-2
6-3 6-2
6-3 6-4
7-5 6-4
6-3 6-7(5) 6-3
6-4 6-7(5) 7-6(4)
6-7(5) 6-4 7-6(7)
4-6 6-3 6-4
6-4 6-7(4) 0-0 RET
7-5 6-1
6-3 6-2
6-4 6-4
6-7(5) 7-5 7-6(1)
7-6(8) 6-2
7-6(5) 7-6(4)
4-6 7-5 6-2
6-7(3) 7-6(6) 6-4
6-3 7-5
5-7 6-3 6-4
6-3 6-4
6-1 6-4
6-4 6-4
6-4 6-4
3-6 6-3 6-4
6-7(4) 6-2 6-3
6-4 6-0
6-2 6-3
6-3 3-6 7-5
7-5 6-4
7-6(5) 6-4
6-4 6-2
6-4 2-6 7-6(2)
6-4 6-4
4-6 6-4 6-3
6-7(5) 6-3 6-4
5-7 6-3 6-4
6-1 6-7(3) 6-1
6-3 6-2
7-6(4) 6-4
6-4 6-1
6-3 6-4
6-4 6-4
4-6 6-2 6-3
4-6 7-5 6-3
6-4 1-6 6-2
6-4 6-4
6-4 6-4
6-3 7-6(4)
6-4 7-6(8)
7-5 7-5
6-3 7-6(4)
6-7(5) 7-6(2) 7-6(2)
6-7(4) 7-6(5) 7-6(2)
7-5 6-3
6-4 6-3
7-6(2) 2-6 6-2
6-4 6-0
6-4 6-4
6-3 4-6 6-1
6-3 6-2
7-5 6-1
6-7(5) 6-3 7-6(8)
3-6 7-6(4) 7-6(7)
6-1 6-2
6-2 6-3
6-4 7-5
6-2 6-4
6-7(5) 6-2 6-1
4-6 6-4 6-3
7-6(2) 6-4
6-2 6-1
6-1 6-4
6-3 6-7(4) 7-5
6-4 6-4
6-4 7-5
6

In [54]:
### Score Reader
def score_reader(score):
    w_games_won = 0
    l_games_won = 0
    tiebreaks = 0
    tiebreak_points = []
    num_sets = 1

    i = 1
    for num in score[1:-1]:
        if num == '-':
            w_games_won += int(score[i-1])
            l_games_won += int(score[i+1])
        if num == '(':
            tiebreaks += 1
            if num[i-1] == '7':
                loser = 'w'
            else:
                loser = 'l'
            tiebreak_points.append(loser, score[i+1])
        if num == ' ':
            num_sets += 1
        i += 1

    total_games = w_games_won + l_games_won
    return w_games_won, l_games_won, total_games, tiebreaks, tiebreak_points, num_sets

In [55]:
print(score_reader('6-1 6-4'))

0


ValueError: invalid literal for int() with base 10: '-'

In [ ]:

### Service Games Won